# Mount and import dataset from Google Cloud Platform (GCP)

In [ ]:
GCP_PROJECT_ID = 'euphoric-grin-335812'
PROJECT_NUMBER = '569090358106'

In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


In [ ]:
BUCKET_NAME = 'bucket_name'

GCS_BUCKET = f'gs://{BUCKET_NAME}'
!gsutil mb -p $GCP_PROJECT_ID $GCS_BUCKET

Creating gs://fish_size_project_data/...
ServiceException: 409 A Cloud Storage bucket named 'fish_size_project_data' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
!gsutil du -s -h gs://{BUCKET_NAME}


59.39 GiB    gs://fish_size_project_data


In [ ]:
# install gcsfuse on colab.
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
40 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 12.0 MB of archives.
After this operation, 28.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.40.0_amd64.deb ...
Unpacking gcsfuse (0.40.0) ...
Setting up gcsfuse (0.40.0) ...


In [ ]:
!mkdir deeper_data
!gcsfuse --implicit-dirs fish_size_project_data deeper_data


2022/04/12 14:00:29.920792 Start gcsfuse/0.40.0 (Go version go1.17.6) for app "" using mount point: /content/deeper_data
2022/04/12 14:00:29.936350 Opening GCS connection...
2022/04/12 14:00:30.129538 Mounting file system "fish_size_project_data"...
2022/04/12 14:00:30.165177 File system has been successfully mounted.


# Get image dimensions

In [ ]:
# from PIL import Image

# im = Image.open('/content/deeper_data/MonthlyDeeperData/2021-06-01/images/000ce93b-5f7a-41ff-acfb-66a4603c6698.png')
# width, height = im.size
# print("width:", width, ", height:", height, ", image size:", im.size)


width: 960 , height: 1280 , image size: (960, 1280)


In [ ]:
import tensorflow as tf

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
import os
import csv


# search for files in 'images' dir
files_dir = os.getcwd() + '/deeper_data/annotated_images_pilot_study'
files = os.listdir(files_dir)

# loop over files, print prediction if it is an image
for f in files:
  if f.lower().endswith(('.jpg')):
    images_path = files_dir + '/' + f
    
    # load images
    img = load_img(images_path)

    # get image height and width
    image_h = img.shape[0:1]
      
    image_w = img.shape[1:2]

    image_path, image_name = os.path.split(images_path)
    
    with open('images_dimensions_pilot_study.csv', 'a') as csvfile:
      fieldnames = ['filename', 'image_height', 'image_width']
      # print(fieldnames)
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames) 
      print(writer)
      writer.writerow({'filename':image_name, 'image_height':image_h, 'image_width': image_w})

   


In [ ]:
import pandas as pd
annotations_formated = pd.read_csv('via_project_annotations_4-5-6-7-01-pilot_formated.csv')

In [ ]:
annotations_formated.head(6)

,filename,x_min,y_min,width,height,species
0,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,355.000000,400.000000,229.000000,406.000000,Abramis brama
1,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,265.127794,41.565881,356.219219,1028.560309,Abramis brama
2,2d68c955-a417-4b2e-857d-efb51013724d.jpg,140.000000,148.000000,882.000000,442.000000,Abramis brama
3,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,394.000000,536.000000,350.000000,221.000000,Abramis brama
4,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,372.000000,679.000000,309.000000,138.000000,Abramis brama
5,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,411.000000,353.000000,380.000000,270.000000,Abramis brama


In [ ]:
images_dimensions = pd.read_csv('images_dimensions_pilot_study_6spp.csv')
images_dimensions.head(5)

,filename,image_height,image_width
0,0016c9bf-8824-41c5-8729-b6496a3c3d56.jpg,1280,960
1,001a840b-bf8c-4101-8414-696e18abee50.jpg,1280,962
2,0031550a-647e-4d31-a374-ca0b958e6097.jpg,960,1280
3,00510ead-0914-4f85-bf22-5cae0ed9d506.jpg,960,1280
4,005149a5-c25a-4dfb-b2df-448304076daa.jpg,1280,960


In [ ]:
annotations_dimensions = pd.merge(annotations_formated, images_dimensions, left_on='filename', right_on='filename')
annotations_dimensions[0:3]

,filename,x_min,y_min,width,height,species,image_height,image_width
0,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,355.000000,400.000000,229.000000,406.000000,Abramis brama,1280,960
1,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,265.127794,41.565881,356.219219,1028.560309,Abramis brama,1280,852
2,2d68c955-a417-4b2e-857d-efb51013724d.jpg,140.000000,148.000000,882.000000,442.000000,Abramis brama,623,1280


In [ ]:
print(len(annotations_formated))
print(len(images_dimensions))
print(len(annotations_dimensions))

1930
1615
1871


In [ ]:
x_rel_min = annotations_dimensions['x_min']/annotations_dimensions['image_width']
y_rel_min = annotations_dimensions['y_min']/annotations_dimensions['image_height']
x_rel_max = (annotations_dimensions['x_min'] + annotations_dimensions['width']) /annotations_dimensions['image_width']
y_rel_max = (annotations_dimensions['y_min'] + annotations_dimensions['height']) /annotations_dimensions['image_height']


annotations_dimensions['x_rel_min'] = x_rel_min
annotations_dimensions['y_rel_min'] = y_rel_min
annotations_dimensions['x_rel_max'] = x_rel_max
annotations_dimensions['y_rel_max'] = y_rel_max
annotations_dimensions.head(5)

,filename,x_min,y_min,width,height,species,image_height,image_width,x_rel_min,y_rel_min,x_rel_max,y_rel_max
0,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,355.000000,400.000000,229.000000,406.000000,Abramis brama,1280,960,0.369792,0.312500,0.608333,0.629687
1,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,265.127794,41.565881,356.219219,1028.560309,Abramis brama,1280,852,0.311183,0.032473,0.729281,0.836036
2,2d68c955-a417-4b2e-857d-efb51013724d.jpg,140.000000,148.000000,882.000000,442.000000,Abramis brama,623,1280,0.109375,0.237560,0.798438,0.947030
3,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,394.000000,536.000000,350.000000,221.000000,Abramis brama,1280,960,0.410417,0.418750,0.775000,0.591406
4,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,372.000000,679.000000,309.000000,138.000000,Abramis brama,1280,960,0.387500,0.530469,0.709375,0.638281


In [ ]:
import random
import pandas as pd

def addTrainSplit():
    annotations_dimensions["split"] = [random.choice(['TRAIN','TRAIN', 'TRAIN', 'TRAIN', 'TRAIN', 'TRAIN', 'TRAIN', 'TRAIN', 'TEST', 'VALIDATION']) for filename in annotations_dimensions.filename]
    
addTrainSplit()


In [ ]:
annotations_dimensions.head(5)

,filename,x_min,y_min,width,height,species,image_height,image_width,x_rel_min,y_rel_min,x_rel_max,y_rel_max,split
0,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,355.000000,400.000000,229.000000,406.000000,Abramis brama,1280,960,0.369792,0.312500,0.608333,0.629687,TRAIN
1,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,265.127794,41.565881,356.219219,1028.560309,Abramis brama,1280,852,0.311183,0.032473,0.729281,0.836036,VALIDATION
2,2d68c955-a417-4b2e-857d-efb51013724d.jpg,140.000000,148.000000,882.000000,442.000000,Abramis brama,623,1280,0.109375,0.237560,0.798438,0.947030,TRAIN
3,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,394.000000,536.000000,350.000000,221.000000,Abramis brama,1280,960,0.410417,0.418750,0.775000,0.591406,TRAIN
4,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,372.000000,679.000000,309.000000,138.000000,Abramis brama,1280,960,0.387500,0.530469,0.709375,0.638281,TRAIN


In [ ]:
print(annotations_dimensions['split'].value_counts()["TRAIN"], "Train entries, ",
      annotations_dimensions['split'].value_counts()["TEST"], "Test entries, ",
      annotations_dimensions['split'].value_counts()["VALIDATION"], "Validation entries")


1508 Train entries,  176 Test entries,  187 Validation entries


# Format dataframe (TensorFlow format) 

In [ ]:
annotations_dimensions["empty"] = ""
annotations_dimensions.head(5)

,filename,x_min,y_min,width,height,species,image_height,image_width,x_rel_min,y_rel_min,x_rel_max,y_rel_max,split,empty
0,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,355.000000,400.000000,229.000000,406.000000,Abramis brama,1280,960,0.369792,0.312500,0.608333,0.629687,TRAIN,
1,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,265.127794,41.565881,356.219219,1028.560309,Abramis brama,1280,852,0.311183,0.032473,0.729281,0.836036,VALIDATION,
2,2d68c955-a417-4b2e-857d-efb51013724d.jpg,140.000000,148.000000,882.000000,442.000000,Abramis brama,623,1280,0.109375,0.237560,0.798438,0.947030,TRAIN,
3,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,394.000000,536.000000,350.000000,221.000000,Abramis brama,1280,960,0.410417,0.418750,0.775000,0.591406,TRAIN,
4,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,372.000000,679.000000,309.000000,138.000000,Abramis brama,1280,960,0.387500,0.530469,0.709375,0.638281,TRAIN,


In [ ]:
annotations_dataset = annotations_dimensions[["split", "filename", "species", "x_rel_min", "y_rel_min", "empty", "empty","x_rel_max", "y_rel_max", "empty",  "empty"]]
annotations_dataset.head(5)

,split,filename,species,x_rel_min,y_rel_min,empty,empty,x_rel_max,y_rel_max,empty,empty
0,TRAIN,0d4b0d8d-ccbb-42f1-bc48-291594bffda0.jpg,Abramis brama,0.369792,0.312500,,,0.608333,0.629687,,
1,VALIDATION,2b210c87-6278-4220-8a8e-ee0d46b10758.jpg,Abramis brama,0.311183,0.032473,,,0.729281,0.836036,,
2,TRAIN,2d68c955-a417-4b2e-857d-efb51013724d.jpg,Abramis brama,0.109375,0.237560,,,0.798438,0.947030,,
3,TRAIN,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,Abramis brama,0.410417,0.418750,,,0.775000,0.591406,,
4,TRAIN,4b5cb558-ac23-45b1-b088-e35bed2d6eff.jpg,Abramis brama,0.387500,0.530469,,,0.709375,0.638281,,


In [ ]:
annotations_dataset.to_csv('annotations_for_model_6spp.csv', index=False)  